In [ ]:
!python --version

# MySQL connection with jupyter notebook

### install mysql-connector-python

In [ ]:
#mysql connection with python
!conda install -c anaconda mysql-connector-python -y

### Connection with mysql

In [ ]:
try:
    import mysql.connector as sql
    #connection to SQL
    conn = sql.connect(
        host='localhost', 
        user='root', 
        password='cap4770',
        database= 'cap4770',  
        use_pure = True)
    
    cursor = conn.cursor(buffered=True)
    cursor.execute("use cap4770")
    
    # Check what tables are available
    cursor.execute("show tables")
    tables = cursor.fetchall()
    print("Available tables in the database:")
    for table in tables:
        print(f"- {table[0]}")
    
except Exception as e:
    print(f"Database connection error: {e}")
    print("Will work with CSV file instead.")

In [ ]:
# install numpy, pandas and matplotlib using pip at terminal

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Storm Events Data Analysis

Let's explore the storm events data from the CSV file and database.

In [ ]:
# If the data has been uploaded to the database, query it from there
# First, let's reconnect to the database
try:
    import mysql.connector as sql
    conn = sql.connect(
        host='localhost', 
        user='root', 
        password='cap4770',
        database= 'cap4770',  
        use_pure = True)
    
    cursor = conn.cursor(buffered=True)
    cursor.execute("use cap4770")
    
    # Check what tables are available
    cursor.execute("show tables")
    tables = cursor.fetchall()
    print("Available tables in the database:")
    for table in tables:
        print(f"- {table[0]}")
    
except Exception as e:
    print(f"Database connection error: {e}")
    print("Will work with CSV file instead.")

In [ ]:
# Query storm events data from database (assuming table name is 'storm_events' or similar)
# You may need to adjust the table name based on how you uploaded the data

try:
    # Try common table names for storm events data - add your table name here
    possible_table_names = ['storm_events', 'stormevents', 'storm_data', 'StormEvents_details', 'cap4']
    
    storm_table_name = None
    for table_name in possible_table_names:
        try:
            cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
            count = cursor.fetchone()[0]
            storm_table_name = table_name
            print(f"Found storm events table: {table_name} with {count} records")
            break
        except:
            continue
    
    if storm_table_name:
        # Query the first 10 records from the database
        query = f"SELECT * FROM {storm_table_name} LIMIT 10"
        storm_df_db = pd.read_sql(query, conn)
        print(f"\nData from database table '{storm_table_name}':")
        print(f"Shape: {storm_df_db.shape}")
        storm_df_db.head()
    else:
        print("Storm events table not found in database. Please check the table name.")
        
except Exception as e:
    print(f"Error querying database: {e}")
    print("Working with CSV file data instead.")

In [ ]:

#searches database for the cost of damages from each weather event type and adds them together. Typecasts the damage_property and damage_crops
query = f"SELECT event_type, SUM(CASE WHEN damage_property LIKE '%k' THEN CAST(REPLACE(damage_property, 'k', '') AS DECIMAL(10,2)) * 1000 WHEN damage_property LIKE '%M' THEN CAST(REPLACE(damage_property, 'M', '') AS DECIMAL(10,2)) * 1000000 WHEN damage_property LIKE '%B' THEN CAST(REPLACE(damage_property, 'B', '') AS DECIMAL(10,2)) * 1000000000 ELSE CAST(damage_property AS DECIMAL(10,2)) END + CASE WHEN damage_crops LIKE '%k' THEN CAST(REPLACE(damage_crops, 'k', '') AS DECIMAL(10,2)) * 1000 WHEN damage_crops LIKE '%M' THEN CAST(REPLACE(damage_crops, 'M', '') AS DECIMAL(10,2)) * 1000000 WHEN damage_crops LIKE '%B' THEN CAST(REPLACE(damage_crops, 'B', '') AS DECIMAL(10,2)) * 1000000000 ELSE CAST(damage_crops AS DECIMAL(10,2)) END) AS total FROM {storm_table_name} GROUP BY event_type"
damage_db = pd.read_sql(query, conn)
# Sort by the 'total' column in descending order
damage_db = damage_db.sort_values(by='total', ascending=False)
#Formats cost into a readable format
pd.options.display.float_format = '${:,.0f}'.format
print(damage_db)
